# Overview
This notebook introduces the basics of object-oriented programming in python by developing an example of a two-dimensional world containing objects that have locations and that can move about in the world.

# Introducing the Vector class
Everything will be built on top of the `Vector` class, which enables our objects to have things like locations and velocities. So we begin by defining that first. We'll decide to make instances of class `Vector` immutable, which means that, once created, their (x,y) coordinates cannot be changed. This is a design choice. We didn't need to do it this way, but it makes sense for this class and it also will help illustrate some useful python concepts, such as the `@property` decorator.

Review the class definition and the examples of its use below, and then we'll discuss what is going on in more detail. Please read all of the comments, i.e., those that begin with a hash-mark, `#`, as they contain instructional material. 

In [1]:
class Vector(object):
    """
    Represents a two-dimensional vector, something with an x-component and a y-component.
    
    Knows how to perform basic vector operations on itself and on other vectors, such as
    scaling and addition.
    """
    def __init__(self, x, y):
        # We use underscores in the names to indicate that these member variables are private.
        self._x = x
        self._y = y
        
    # Here we supply getter methods that give clients read-only access to the x and y coordinates.
    def x(self):
        return self._x
    
    def y(self):
        return self._y
    
# Examples
print("Creating a vector")
v = Vector(1.2, 3.4)
print("v =", v)

print()
print("Accessing its components")
print("v.x() =", v.x())
print("v.y() =", v.y())
    
print()
print("Note that in python we can always get around read-only restrictions if we violate protocol.")
print("If we really want to change the value of v._x, we can do it.")
v._x = 5.6
print("v.x() =", v.x())
        

Creating a vector
v = <__main__.Vector object at 0x11186ada0>

Accessing its components
v.x() = 1.2
v.y() = 3.4

Note that in python we can always get around read-only restrictions if we violate protocol.
If we really want to change the value of v._x, we can do it.
v.x() = 5.6


There are several things to note about the code example above. 

## Inherit from class object
Note that our base class `Vector` inherits from the class `object`. Our base classes should always do this.


## Make things private using underscores
In python, by convention, whenever a variable, function, or method name begins with an underscore, it is private. That means that client code should not attempt to read it, write it, or call it, as the case may be. It is good form, when defining a class, to clearly demarcate its public interface. Those member variables and methods that are not named with leading underscores are deemed part of its public interface. Client code is free to both read and write public member variables, and it is free to call public methods. Everything else is off limits.

We don't want client code to be able to change the values of a vector's x and y components, but we do want client code to be able to read them. Naming our member variables `self._x` and `self._y` prevents client code from modifying them. In order to grant read-access to them, we created the public methods x() and y().




# Make read-only properties with @property
There is nothing wrong with our first draft of `Vector`, but it does seem a bit of a shame that we have to access a vector's x- and y-components via getter methods, e.g., `v.x()` and `v.y()`, rather than just accessing them like member variables, e.g., `v.x` and `v.y`. If `x` and `y` were public member variables, we'd be able to access them directly. It's only because they're private that we had to resort to getter methods.

In python, there is an elegant way to allow client code to use the expressions `v.x` and `v.y` to read the x- and y-components of a vector, as though they were member variables (even though they're not), while still disallowing writing. This is done with the `@property` decorator. The version below is identical to the version above, except that the two getter methods now have the `@property` decorator above their definitions. Take a look at the slightly modified version of the code below and then we'll discuss it in detail.

In [2]:
# Modify the Vector class to use @property for access to its x- and y-components.
class Vector(object):
    """
    Represents a two-dimensional vector, something with an x-component and a y-component.
    
    Knows how to perform basic vector operations on itself and on other vectors, such as
    scaling and addition.
    """
    def __init__(self, x, y):
        # We use underscores in the names to indicate that these member variables are private.
        self._x = x
        self._y = y
        
    # Here we supply properties that give clients read-only access to the x and y coordinates.
    @property
    def x(self):
        return self._x
    
    # Same as above, but for the y-component.
    @property
    def y(self):
        return self._y
    
# Examples
print("Creating a vector")
v = Vector(1.2, 3.4)
print("v =", v)

print()
print("Accessing its components")
print("v.x =", v.x)
print("v.y =", v.y)

print()
print("Note that attempting to set its components raises an error!")
try:
    v.x = 5.6
except:
    print("Caught an error while trying to set v.x to 5.6")
    
print()
print("Note that in python we can always get around such restrictions if we violate protocol.")
print("If we really want to change the value of v.x, we can do it like this:")
v._x = 5.6
print("v.x =", v.x)

Creating a vector
v = <__main__.Vector object at 0x111910cc0>

Accessing its components
v.x = 1.2
v.y = 3.4

Note that attempting to set its components raises an error!
Caught an error while trying to set v.x to 5.6

Note that in python we can always get around such restrictions if we violate protocol.
If we really want to change the value of v.x, we can do it like this:
v.x = 5.6


As we can see from the modified example section above, it is now possible to refer to the components of the vector `v` as `v.x` and `v.y` instead of the clunkier `v.x()` and `v.y()`. The methods we defined are still being called under the hood. Only the syntax has changed. This allows for a clearer class definition, because the x- and y-components feel like member variables, rather than methods. Conceptually, they truly are member variables. We only interpose the methods to protect them from being written to. The `@property` decorator allows us to treat them as read-only member variables. (Of course, we can still write to `v._x` and `v._y` if we are willing to break  convention.)

It is beyond the scope of this tutorial to describe in detail how `@property` works under the hood, but let's talk about when one can use it. Whenever we have a method that only takes `self` as a single argument, we can decorate the method by putting `@property` above its definition. Doing so will allow us to "call" the method without employing open and close parentheses, e.g., via `my_obj.foo` instead of via `my_obj.foo()`. Things will look syntactically as though we're just reading a member variable. Don't try decorating with `@property` a method that takes more than `self` as arguments. It won't work, and, in fact, it would be meaningless, since we would have no way to pass the second arg to the method without using parentheses.

We should also note that, having decorated the method `foo()` with `@property`, we can no longer call this method via `my_obj.foo()`. We *must* omit the parens and "call" it like `my_obj.foo`.

# Use @property to turn lazy calculations into properties
We can use `@property` for more than just forcing member variables to be read-only. We can define any function of our object's member variables to be a property. This is usually a desireable thing to do when the property should always immediately change to reflect the current values of the variables from which it is calculated. Below is a typical example in which we define a `Rectangle` class with an `area` property.

In [3]:
# Example of a class with a lazily calculated property.
class Rectangle(object):
    def __init__(self, width, height):
        self.width  = width
        self.height = height
    
    @property
    def area(self):
        return self.width * self.height
    
# Examples
rectangle = Rectangle(3,4)
print("The current area of rectangle is", rectangle.area)

rectangle.width = 10
print("We just changed rectangle.width to 10")

print("The area of rectangle is now", rectangle.area)

The current area of rectangle is 12
We just changed rectangle.width to 10
The area of rectangle is now 40


By defining `area` as a property above, we ensure that it always reflects the area as a function of the current width and height of the rectangle. Note that this is a "lazy" calculation. We don't update the area whenever the width or height of the rectangle changes (which would be an "eager" calculation). We just calculate it whenever it is referenced. That is, every time `rectangle.area` is referenced, the method `Rectangle.area()` is called. The resulting value of area is not cached anywhere by the python system.

# Use @property to maintain consistency and to validate.
In the example above, we used lazy calculation to calculate the area of a rectangle. That is the appropriate way to do things in this case, because the calculation is quick, so we don't need to cache the result. For the sake of argument, though, let's suppose that the area calculation were time intensive and that we therefore only want to calculate it when absolutely necessary, i.e., whenever the width or height of the rectangle changes, caching the result for future use (such a setup is called "eager" calculation). And suppose, further, we want to prevent client code from assigning non-positive values for width and height, since these are invalid. We can use `@property` to do both these things. The mechanism involves an aspect of `@property` that we have not seen yet. It allows us to intercept any changes to the member variables of Rectangle, running arbitrary code before changing their values. Take a look at the code below, and then we'll discuss it in detail.


In [4]:
class Rectangle(object):
    def __init__(self, width, height):
        # These private member variables are maintained by getters and setters.
        self._width  = 1.0  # We have to set this to a valid value for the setter on line 12 to work.
        self._height = 1.0  # We have to set this to a valid value for the setter on line 12 to work.
        self._area   = None
        
        # We could have set the member variables above directly, but by invoking the getters and setters below,
        # we ensure that Rectangle is not created with invalid width or height. Note also that, although there
        # seems to be no explicit calculation of area done below, in fact it gets done twice implicitly:
        # once when we set the width, and again when we set the height.
        self.width  = width 
        self.height = height
        
    @property
    def width(self):
        return self._width
    
    # The method name "width" below is required. It must match the name of the property defined above.
    @width.setter
    def width(self, new_width):
        assert new_width > 0, "width must be greater than 0"
        self._width = new_width
        self._area  = self._calc_area()
        
    
    @property
    def height(self):
        return self._height
    
    # The method name "height" below is required. It must match the name of the property defined above.
    @height.setter
    def height(self, new_height):
        assert new_height > 0, "height must be greater than 0"
        self._height = new_height
        self._area   = self._calc_area()
    
    # Note that we don't define a setter for area. It is read-only.
    @property
    def area(self):
        return self._area
    
    def _calc_area(self):
        return self.width * self.height
    
# Examples
rectangle = Rectangle(3,4)
print("The current area of rectangle is", rectangle.area)

rectangle.width = 10
print("We just changed rectangle.width to 10")

print("The area of rectangle is now", rectangle.area)

The current area of rectangle is 12
We just changed rectangle.width to 10
The area of rectangle is now 40


On line 15 of the code above, when we use `@property` to decorate the `width()` method, it automatically generates a new decorator called `@width.setter`. When we use this decorator to decorate the method on line 21--which also _must_ be named `width`--we are telling python how to handle updates to the `width` property. Whenever any code invokes `rectangle.width = <blah>`, as on line 50 above, it is the setter method on line 21 that is called. The new value proposed for the `width` property--in this case 10--is passed in as an argument to the method. The method is solely responsible for handling the update. In this case, it not only updates the member variable `_width` on line 23, but it also makes sure the new proposed value is legal, raising an exception if it is not, and it further updates the `_area` member variable, whose value has also changed as a result of the change to `width`. If the setter fails to update the `_width` member variable, then the width property remains unchanged! Although this is legal, it will result in confusing code, since readers of the code on line 50 will naturally assume that it either changed the width as specified or raised an error.

Note that the setter methods take an extra argument than the getter methods above. A getter method must always take just `self` as an argument, and a setter method must always take both `self` and `new_value` as arguments. (The names of course are arbitrary. We can name formal parameters any names we like. Even `self` is merely conventional, not required. There is nothing special about the name `self`.)

Note also that the names of setter methods are required to match their property names, as illustrated on lines 21 and 33 above.

# Properties Summarized
In sum, the `@property` decorator allows us to create something called a property. A property looks like a member variable, syntactically, but is read and updated via getter and setter methods that we define. When we decorate a method with `@property`, we are defining the getter method for a property of the same name as the method. By so doing, we automatically generate a new decorator to be the setter. If the name of the property is `foo`, then the name of the setter decorator becomes `@foo.setter`. We can use this decorator to define a setter method for the property `foo` if we wish. If we don't define a setter method, then setting the property remains an illegal operation and the property remains read-only.